Finetuning IndicBART for summarization in malayalam

In [1]:
!sudo apt install git-lfs -y
!sudo apt install build-essential cmake python3-dev -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
python3-dev is already the newest version (3.10.6-1~22.04.1).
python3-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [1]:
!git clone https://github.com/prajdabre/yanmtt

Cloning into 'yanmtt'...
remote: Enumerating objects: 2837, done.
remote: Counting objects: 100% (344/344), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 2837 (delta 222), reused 336 (delta 218), pack-reused 2493 (from 1)
Receiving objects: 100% (2837/2837), 45.60 MiB | 15.66 MiB/s, done.
Resolving deltas: 100% (1330/1330), done.


In [2]:
%cd yanmtt

/content/yanmtt


In [3]:
%ls

average_checkpoints.py         interface/
common_utils.py                LICENSE
create_autoconfig.py           pretrain_nmt.py
create_autotokenizer.sh        README.md
decode_nmt.py                  train_nmt.py
examples/                      transformers/
gpu_blocker.py                 yanmtt_pip_requirements.txt
installation_instructions.txt


In [4]:
!pip install -r yanmtt_pip_requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.4/392.4 kB 14.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 39.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 

In [5]:
%cd transformers

/content/yanmtt/transformers


In [6]:
!python setup.py install

running install
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https

In [7]:
%cd ..

/content/yanmtt


# Loading Data


In [8]:
import pandas as pd

df = pd.read_csv("hf://datasets/rahulraj2k16/Social-Sum-Mal/baseline_1_Social-sum-Mal.csv")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
df.head()

,Unnamed: 0,chapter_no,para_no,heading,input,long_summary,extreme_summary,query,answer_summary,topic
0,1.0,1,1.0,വിപ്ലവങ്ങളുടെ ആരംഭം,എല്ലായിടത്തും എല്ലാവർക്കും സ്വാതന്ത്ര്യം ലഭിച്...,പല നാടുകളിലും ജനതയ്ക്ക് സ്വാതന്ത്ര്യം ലഭിച്ചിര...,പല നാടുകളിലെയും ആളുകൾ സ്വാതന്ത്ര്യത്തിന് വേണ്ട...,വിപ്ലവം എന്നാൽ എന്ത് ?,പല നാടുകളിലും ജനതയ്ക്ക് സ്വാതന്ത്ര്യം ലഭിച്ചിര...,word history
1,2.0,1,2.0,ജ്ഞാനോദയം യൂറോപ്പിൽ,യുക്തിരഹിതമായ വിശ്വാസങ്ങളെയും സമ്പ്രദായങ്ങളെയു...,ജ്ഞാനോദയ ചിന്തകർ പ്രചരിപ്പിച്ച നവോത്ഥാന ആശയങ്ങ...,ജ്ഞാനോദയ ചിന്തകർ പ്രചരിപ്പിച്ച നവോത്ഥാന ആശയങ്ങ...,ജ്ഞാനോദയ പ്രസ്ഥാനങ്ങൾ യൂറോപ്യൻ സമൂഹത്തിൽ ചെലുത...,ജ്ഞാനോദയ ചിന്തകർ പ്രചരിപ്പിച്ച നവോത്ഥാന ആശയങ്ങ...,word history
2,3.0,1,1.0,അമേരിക്കയിലെ കുടിയേറ്റം,1492 ൽ സ്പാനിഷ് ഗവൺമെന്റിന്റെ നാവികനായി വടക്കേ...,1492 ൽ സ്പാനിഷ് ഗവൺമെന്റിന്റെ നാവികനായി വടക്ക...,ഇംഗ്ലണ്ടിലെ രാജാവിന്റെ മതപീഡനത്തെത്തുടർന്ന് പത...,അമേരിക്കൻ കുടിയേറ്റം വിശദീകരിക്കുക ?,ഇംഗ്ലണ്ടിലെ രാജാവിന്റെ മതപീഡനത്തെത്തുടർന്ന് പത...,word history
3,4.0,1,1.0,മെർക്കന്റലിസ്റ്റ് നിയമങ്ങൾ,കോളനികളിൽ നിന്നോ കോളനികളിലേക്കോ സാധനങ്ങൾ കൊണ്ട...,ചരക്ക് നീക്കത്തിന് ഇംഗ്ലീഷ് അംഗീകൃത കപ്പലുകൾ ഉ...,കോളനികളുടെ പൂർണ നിയന്ത്രണം ഏറ്റെടുക്കും വിധം ഇ...,മെർക്കന്റലിസ്റ്റ് നിയമങ്ങൾ വ്യക്തമാക്കുക?,ചരക്ക് നീക്കത്തിന് ഇംഗ്ലീഷ് അംഗീകൃത കപ്പലുകൾ ഉ...,word history
4,5.0,1,2.0,ബോസ്റ്റൺ ടീപാർട്ടി,ഇംഗ്ലീഷ് ഗവൺമെന്റ് തേയിലയുടെ മേൽ ഉയർന്ന നികുതി...,ഇംഗ്ലീഷ് ഗവർമെന്റിന്റെ ഉയർന്ന തേയില നികുതി അമേ...,ഇംഗ്ലീഷ് ഗവൺമെന്റിന്റെ അമിത തേയില ചുങ്കത്തിനെ...,ബോസ്റ്റൺ ടീപാർട്ടി എന്തെന്ന് വിശദീകരിക്കുക?,ഇംഗ്ലീഷ് ഗവർമെന്റിന്റെ ഉയർന്ന തേയില നികുതി അമേ...,word history


In [10]:
df = df[['input', 'extreme_summary']].dropna()
df['input'] = df['input'].astype(str).str.strip()
df['extreme_summary'] = df['extreme_summary'].astype(str).str.strip()

# Remove empty rows if any
df = df[(df['input'] != "") & (df['extreme_summary'] != "")]

Splitting

In [11]:
from sklearn.model_selection import train_test_split

# First split: 80% train, 20% temp (val + test)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)

# Second split: equally split temp_df into validation and test (each 10% total)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


In [12]:
# Function to save splits to .src and .tgt files
def save_split(df, prefix):
    with open(f"{prefix}.ml.src", "w", encoding="utf-8") as src_file, \
         open(f"{prefix}.ml.tgt", "w", encoding="utf-8") as tgt_file:
        for _, row in df.iterrows():
            src_file.write(row['input'] + "\n")
            tgt_file.write(row['extreme_summary'] + "\n")

In [13]:
save_split(train_df, "train")
save_split(val_df, "dev")
save_split(test_df, "test")

In [14]:
print(f"Train size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")
print(f"Test size: {len(test_df)}")

Train size: 1600
Validation size: 200
Test size: 200


In [15]:
!ls -lh *.src *.tgt

-rw-r--r-- 1 root root 251K Jun  9 14:40 dev.ml.src
-rw-r--r-- 1 root root  55K Jun  9 14:40 dev.ml.tgt
-rw-r--r-- 1 root root 252K Jun  9 14:40 test.ml.src
-rw-r--r-- 1 root root  60K Jun  9 14:40 test.ml.tgt
-rw-r--r-- 1 root root 2.0M Jun  9 14:40 train.ml.src
-rw-r--r-- 1 root root 463K Jun  9 14:40 train.ml.tgt


In [16]:
# Take first 10 lines from train files and save as 'train_10.ml.src' and 'train_10.ml.tgt'
def save_first_n_lines(src_file, tgt_file, prefix, n=10,):
    with open(src_file, 'r', encoding='utf-8') as fsrc, \
         open(tgt_file, 'r', encoding='utf-8') as ftgt:
        src_lines = [next(fsrc).strip() for _ in range(n)]
        tgt_lines = [next(ftgt).strip() for _ in range(n)]

    with open(f"{prefix}_10.ml.src", 'w', encoding='utf-8') as fsrc_out, \
         open(f"{prefix}_10.ml.tgt", 'w', encoding='utf-8') as ftgt_out:
        for s, t in zip(src_lines, tgt_lines):
            fsrc_out.write(s + "\n")
            ftgt_out.write(t + "\n")

save_first_n_lines("train.ml.src", "train.ml.tgt", "train",10)


In [17]:
save_first_n_lines("dev.ml.src", "dev.ml.tgt", "dev", 10)

In [18]:
%ls

average_checkpoints.py         LICENSE
common_utils.py                pretrain_nmt.py
create_autoconfig.py           README.md
create_autotokenizer.sh        test.ml.src
decode_nmt.py                  test.ml.tgt
dev_10.ml.src                  train_10.ml.src
dev_10.ml.tgt                  train_10.ml.tgt
dev.ml.src                     train.ml.src
dev.ml.tgt                     train.ml.tgt
examples/                      train_nmt.py
gpu_blocker.py                 transformers/
installation_instructions.txt  yanmtt_pip_requirements.txt
interface/


In [19]:
import os

files_to_patch = [
    "/content/yanmtt/decode_nmt.py",
    "/content/yanmtt/pretrain_nmt.py",
    "/content/yanmtt/common_utils.py",
    "/content/yanmtt/train_nmt.py",
]

for file_path in files_to_patch:
    if not os.path.exists(file_path):
        print(f"❌ File not found: {file_path}")
        continue

    with open(file_path, "r", encoding="utf-8") as f:
        code = f.read()

    if "from transformers import AdamW" in code:
        code = code.replace("from transformers import AdamW", "from torch.optim import AdamW")
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(code)
        print(f"✅ Patched {file_path}")
    else:
        print(f"ℹ️ No change needed in {file_path}")


✅ Patched /content/yanmtt/decode_nmt.py
✅ Patched /content/yanmtt/pretrain_nmt.py
✅ Patched /content/yanmtt/common_utils.py
✅ Patched /content/yanmtt/train_nmt.py


In [63]:
!grep -r "from transformers import AdamW" /content/yanmtt


/content/yanmtt/transformers/examples/research_projects/longform-qa/eli5_utils.py:from transformers import AdamW, AutoModel, AutoModelForSeq2SeqLM, AutoTokenizer, get_linear_schedule_with_warmup
/content/yanmtt/transformers/docs/source/training.rst:    from transformers import AdamW
/content/yanmtt/transformers/docs/source/custom_datasets.rst:    from transformers import AdamW


In [20]:
!pip install -U transformers==4.36.2 tokenizers==0.15.2

DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/transformers-4.3.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 118.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is 

In [21]:
!pip install sacrebleu

DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/transformers-4.3.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.0 MB/s eta 0:00:00
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.8 MB/s eta 0:00:00


In [22]:
!pip install rouge-score

DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/transformers-4.3.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Using cached rouge_score-0.1.2.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8abc6dd3c963a183e2dd69af890df00c9c25ca953f04ca22c4e8f2b549a2c05d
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [37]:
!find /content -name "indicbart_model.ckpt"

In [38]:
!wget https://storage.googleapis.com/indic-nlp-models/v2/indicbart/indicbart_model.ckpt

--2025-06-09 15:36:21--  https://storage.googleapis.com/indic-nlp-models/v2/indicbart/indicbart_model.ckpt
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.203.207, 142.250.98.207, 142.251.107.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.203.207|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-06-09 15:36:21 ERROR 404: Not Found.



In [36]:
!python train_nmt.py \
  --train_slang ml --train_tlang ml \
  --dev_slang ml --dev_tlang ml \
  --train_src train_10.ml.src --train_tgt train_10.ml.tgt \
  --dev_src dev_10.ml.src.ml.src --dev_tgt dev_10.ml.tgt \
  --model_path model_poc.ml \
  --encoder_layers 2 --decoder_layers 2 \
  --label_smoothing 0.1 \
  --dropout 0.1 --attention_dropout 0.1 \
  --activation_dropout 0.1 \
  --encoder_attention_heads 4 --decoder_attention_heads 4 \
  --encoder_ffn_dim 512 --decoder_ffn_dim 512 \
  --d_model 256 \
  --tokenizer_name_or_path albert-base-v2 \
  --warmup_steps 100 \
  --weight_decay 0.00001 \
  --lr 0.001 \
  --max_gradient_clip_value 1.0 \
  --dev_batch_size 4 \
  --shard_files --hard_truncate_length 256 \
  --pretrained_model indicbart_model.ckpt


/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
2025-06-09 15:28:19.612098: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749482899.635538   13532

In [ ]:
!tail -f train.log